<a href="https://colab.research.google.com/github/midhunjmes/midhun-26-02-24/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

hi


In [1]:
pip install presidio-analyzer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 7.8 MB/s eta 0:00:00


In [3]:
pip install presidio_anonymizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 18.7 MB/s eta 0:00:00


In [2]:
!pip install spacy


In [4]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
!pip install geotext spacy

In [23]:
from presidio_analyzer import AnalyzerEngine,PatternRecognizer,Pattern
from presidio_anonymizer import AnonymizerEngine
from presidio_analyzer.nlp_engine import NlpEngineProvider
import re
#from geotext import GeoText

In [24]:
nlp_config = {
    "nlp_engine_name": "spacy",
    "models": [{"lang_code": "en", "model_name": "en_core_web_lg"}]
}
street_address_pattern = Pattern(
    name="street_address_pattern",
    regex=r"\d+\s[A-Za-z]+\s(?:Street|St|Avenue|Ave|Road|Rd|Boulevard|Blvd|Lane|Ln|Drive|Dr|Court|Ct|Place|Pl|Square|Sq)\b",
    score=0.85
)
street_recognizer = PatternRecognizer(
    supported_entity="STREET_ADDRESS",  # Custom entity type
    patterns=[street_address_pattern]
)



provider = NlpEngineProvider(nlp_configuration=nlp_config)
nlp_engine = provider.create_engine()

analyzer=AnalyzerEngine(nlp_engine=nlp_engine)
anonymizer=AnonymizerEngine()
analyzer.registry.add_recognizer(street_recognizer)

text='''John Smith and his friend Sarah Johnson recently moved to 789 Elm Street, San Francisco, CA.
John’s email is john.smith@example.com, and Sarah’s email is sarah.johnson@email.net.
They both work at TechCorp, located at 456 Silicon Ave, San Jose, CA.

If you need to reach John, you can call him at +1-415-555-1234.
Sarah’s phone number is (408) 555-5678.

Their friend David Brown, who lives in New York, NY, often visits them.
David’s contact email is david.brown@business.org, and his phone number is 917-555-7890.

Last summer, they took a trip to Los Angeles and stayed at the Grand Sunset Hotel on 12th Avenue.
For hotel reservations, they called +1 310-555-4321.

During the trip, they met Emily White, a researcher from Chicago, IL.
Emily's email is emily.white@university.edu, and her office number is (312) 555-2468.

Later, they traveled to Miami, FL, where they stayed at 234 Ocean Drive, and visited their old college professor, Dr. Robert Williams, whose number is 786-555-1357.

At the end of their journey, they returned home and continued their work at TechCorp.
They frequently travel between different cities for business meetings in Austin, TX, and Seattle, WA.
'''
texts="i'm midhun and i live in Kulathur."
#results= analyzer.analyze(text=texts,entities=["EMAIL_ADDRESS","PHONE_NUMBER","PERSON","LOCATION","STREET_ADDRESS"],language="en")
# result_text=anonymizer.anonymize(texts,results)
# print(result_text)
results= analyzer.analyze(text=text,entities=["EMAIL_ADDRESS","PHONE_NUMBER","PERSON","LOCATION","STREET_ADDRESS"],language="en")
result_text=anonymizer.anonymize(text,results)
print(result_text)


text: <PERSON> and his friend <PERSON> recently moved to <STREET_ADDRESS>, <LOCATION>, <LOCATION>. 
<PERSON>’s email is <EMAIL_ADDRESS>, and <PERSON>’s email is <EMAIL_ADDRESS>. 
They both work at TechCorp, located at <STREET_ADDRESS>, <LOCATION>, <LOCATION>. 

If you need to reach <PERSON>, you can call him at <PHONE_NUMBER>. 
<PERSON>’s phone number is <PHONE_NUMBER>. 

Their friend <PERSON>, who lives in <LOCATION>, <LOCATION>, often visits them. 
<PERSON>’s contact email is <EMAIL_ADDRESS>, and his phone number is <PHONE_NUMBER>. 

Last summer, they took a trip to <LOCATION> and stayed at the Grand Sunset Hotel on 12th Avenue. 
For hotel reservations, they called <PHONE_NUMBER>.

During the trip, they met <PERSON>, a researcher from <LOCATION>, <LOCATION>. 
Emily's email is <EMAIL_ADDRESS>, and her office number is <PHONE_NUMBER>. 

Later, they traveled to <LOCATION>, <LOCATION>, where they stayed at <STREET_ADDRESS>, and visited their old college professor, Dr. <PERSON>, whose num

In [9]:
import json

In [42]:


street_address_pattern = Pattern(
    name="street_address_pattern",
    regex=r"\d+?,\s[A-Za-z]+\s(?:Street|St|Avenue|Ave|Road|Rd|Boulevard|Blvd|Lane|Ln|Drive|Dr|Court|Ct|Place|Pl|Square|Sq)\b",
    score=0.85
)
street_recognizer = PatternRecognizer(
    supported_entity="STREET_ADDRESS",
    patterns=[street_address_pattern]
)
analyzer.registry.add_recognizer(street_recognizer)
def anonymize_json(data):
    if isinstance(data, dict):
        return {key: anonymize_json(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [anonymize_json(item) for item in data]
    elif isinstance(data, str):
        results = analyzer.analyze(text=data, entities=["PERSON", "EMAIL_ADDRESS", "PHONE_NUMBER", "LOCATION","STREET_ADDRESS"], language="en")
        return anonymizer.anonymize(text=data, analyzer_results=results).text if results else data
    else:
        return data
json_data = {
    "name": "Midhun James",
    "email": "mid@gmail.com",
    "phone": "+911234567890",
    "address": "123 Church Street, Kulathoor",
    "messages": [
        "Hello, this is midhun!",
        "My email is mid@gmail.com and I live in Kochi."
    ]
}
anonymized_data = anonymize_json(json_data)
print(json.dumps(anonymized_data, indent=2))

{
  "name": "<PERSON>",
  "email": "<EMAIL_ADDRESS>",
  "phone": "<PHONE_NUMBER>",
  "address": "<STREET_ADDRESS>, <PERSON>",
  "messages": [
    "Hello, this is <PERSON>!",
    "My email is <EMAIL_ADDRESS> and I live in <LOCATION>."
  ]
}
